## Lab Assignment Three: Extending Logistic Regression 
GROUP MEMBERS:
- **Alex Chen** 
- **Paige Maple** 
- **Sam Valentine**

### Sources
1. ChatGPT (For formatting text and plots)

### Load, Split, and Balance (1.5 points total)

#### Part One (0.5 pts)

#### Part Two (0.5 pts)

#### Part Three (0.5 pts)

### Pre-processing and Initial Modeling (2.5 points total)

#### Part One (0.5 pts)

#### Part Two (0.5 pts)

#### Part Three (0.5 pts)

#### Part Four (1 pts)

### Modeling (5 points total)

#### Part One (1 pts)

#### Part Two (1 pts)

#### Part Three (1 pts)

#### Part Four (2 pts)

### Exceptional Work (1 points total)
One idea (required for 7000 level students):  Implement adaptive momentum (AdaM) in the five layer neural network and quantify the performance compared to other methods.  